In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.quantization
import torchvision
import matplotlib.pyplot as plt
from quartz.utils import get_accuracy, encode_inputs, decode_outputs, plot_output_histograms
import sinabs
import sinabs.layers as sl
import quartz
from copy import deepcopy
from cifar_model import MobileNet

In [ ]:
batch_size = 16
device = 'cuda'

valid_dataset = datasets.CIFAR10(root='./data', train=False, transform=transforms.ToTensor(), download=True)
valid_loader = DataLoader(dataset=valid_dataset, shuffle=False, batch_size=batch_size)
test_loader = DataLoader(dataset=valid_dataset, batch_size=100, shuffle=True)

In [ ]:
percentile = 99.9999
ann = torch.load(f"./cifar-convnet-normalised-{percentile}.pth", map_location=torch.device(device)).eval()
ann;

In [ ]:
short = deepcopy(ann[:7])
snn = quartz.from_torch.from_model(short, t_max=2**5+1, add_spiking_output=False)

In [ ]:
sample_data = next(iter(test_loader))[0]
sample_data.shape

In [ ]:
q_ann = deepcopy(ann)
for exponent in range(4, 8):
    t_max = 2**exponent+1
    def quantize(module, input, output):
        return (output * t_max).round() / t_max

    for module in q_ann.children():
        if isinstance(module, nn.ReLU):
            module.register_forward_hook(quantize)
    q_ann[-1].register_forward_hook(quantize)

    accuracy = get_accuracy(q_ann, valid_loader, device)
    print(f"{t_max} time steps: {round(accuracy, 4)}%")

In [ ]:
sample_data = next(iter(test_loader))[0]
output_layers = [layer for layer in ann.children() if isinstance(layer, nn.ReLU)]
output_layers += [ann[-1]]
# plot_output_histograms(ann, sample_data, )
output_layers;

In [ ]:
sample_data.shape

In [ ]:
get_accuracy(ann, valid_loader, device)

In [ ]:
exponent = 4
snn = quartz.from_torch.from_model(ann, t_max=2**exponent+1, add_spiking_output=True)
snn;

In [ ]:
ann = ann.cuda()

for exponent in range(4, 8):
    t_max = 2**exponent+1
    snn = quartz.from_torch.from_model(ann, t_max=t_max, add_spiking_output=True).to(device).eval()
    snn[-1].rectification = False
    accuracy = get_accuracy(snn, valid_loader, device, t_max=t_max)
    print(f"{t_max} time steps: {round(accuracy, 3)}%")

In [ ]:
t_max = 2**3+1
images, label = next(iter(valid_loader))
spikes = encode_inputs(images, t_max=t_max).to(device)
snn = quartz.from_torch.from_model(ann, t_max=t_max, add_spiking_output=True).to(device).eval()
output_layers = [child for name, child in snn.named_children() if isinstance(child, sl.StatefulLayer)]

plot_output_histograms(snn, spikes, output_layers, t_max=t_max)

In [ ]:
t_max = 2**3+1
def quantize(module, input, output):
    return (output * t_max).int() / t_max

for module in ann.children():
    if isinstance(module, nn.ReLU):
        module.register_forward_hook(quantize)
ann[-1].register_forward_hook(quantize)

param_layers = [child for name, child in ann.named_children() if isinstance(child, (nn.Conv2d, nn.Linear))]
output_layers = [child for name, child in ann.named_children() if isinstance(child, nn.ReLU)]
output_layers += [param_layers[-1]]
plot_output_histograms(ann, images.to(device), output_layers)